# Report 

Здравствуйте.

Я применил алгоритм Simulated Annealing для проблемы Traveling Salesman.
В качестве данных, были поданы координаты городов России, которые сначала подверглись обработке. 

У городов Москва и Санкт-Петербург отсутствовали названия, их я заменил на соответствующие. У города Иннополис количество жителей было записано не правильно. Я поменял строку 96[3] на число 963.
Данные координат городов также подверглись обработке. Я использовал библиотеку Bokeh для визуализации. На вход для того метода визуализации, что выбрал я, нужны были широта и долгота формата EPSG:3857. Для перевода в данный формат я использовал библиотеку pyproj.

Далее я написал функции подсчета расстояния от точки до точки, учитывая сферичность земли. Также я написал функцию подсчета energy distribution
$p^*_{sales}(path) = e^{\frac{-dist(path)}{T}}$.


Следуя алгоритму Simulated Annealing я генерировал новый маршрут, меняя два города местами, сравнивая полученные energy distribution с помощью простого acceptance ratio. Принимая новые значения, или отвергая, в зависимости от U(0,1), которая генерировалась на каждом шагу.
Постепенно снижая температуру T, с шагом ann_rate, я получил желаемый результат.

По ходу тестирования алгоритма, я пытался подобрать лучшие значения для T, ann_rate и даже пытался подобрать random seed. Я пытался запустить алгоритм для T = 25000, 40000 и 60000, ann_rate = 0.99,0.98 и 0.997. Хочу заметить, что понижение ann_rate ниже 0.98 приносило лишь ухудшение работы алгоритма из-за слишком быстрого остывания. Далее перебором пытался найти лучший random seed. Лучшие значения на каждой паре колебались в районе 21 тысячи километров. В итоге лучшие значения, которые мне удалось найти это T=25000, ann_rate = 0.99, a random seed = 24. 

Итоговый результат был 20356.210833 километров. 



# Imports 

In [1]:
%pylab inline
%precision 6

import numpy as np
import pandas as pd
import bokeh 
from math import sin, cos, sqrt, atan2, radians

Populating the interactive namespace from numpy and matplotlib


In [2]:
import bokeh
from bokeh.models import BasicTicker, ColorBar, ColumnDataSource, LinearColorMapper, PrintfTickFormatter, FactorRange, Range1d, ColorBar
from bokeh.transform import transform, linear_cmap, factor_cmap, factor_mark, jitter
from bokeh.plotting import figure, output_file, output_notebook, show
from bokeh.colors import RGB 
from bokeh.palettes import Spectral6, Viridis256
from bokeh.io import show, output_file
from bokeh.core.properties import value
from bokeh.util.hex import hexbin
from bokeh.layouts import column,row,gridplot
from bokeh.tile_providers import CARTODBPOSITRON
from bokeh.models import ColumnDataSource, Range1d, LabelSet, Label
from bokeh.models.glyphs import Line

In [3]:
output_notebook()

Loading BokehJS ...

# Preprocessing 

In [476]:
df = pd.DataFrame.from_csv('cities.csv')

C:\Users\Alexander\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls
  """Entry point for launching an IPython kernel.


In [477]:
df.loc[df.Население=='96[3]', ['Население']] = 963

In [478]:
df.Население = df.Население.astype(int)
df = df.nlargest(30,'Население')

keys = ['Город','Широта','Долгота']
df_city = df[keys]

In [479]:
df_city.loc[101000.0, ['Город']] = 'Москва'
df_city.loc[190000.0, ['Город']] = 'Петербург'

C:\Users\Alexander\Anaconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [480]:
import pyproj

project_projection = pyproj.Proj("+init=EPSG:4326")  # wgs84
google_projection = pyproj.Proj("+init=EPSG:3857")  # default google projection

longitude =  df_city.Долгота.values
latitude = df_city.Широта.values

x, y = pyproj.transform(project_projection, google_projection, longitude, latitude)

print(x, y)

[ 4187880.821473  3374741.923679  9230729.89694   6746572.141361
  4898658.141293  5467070.645262  5577860.021508  8167228.71101
  6835104.209563  4421475.099239  6229201.004283  4955612.075958
  6259960.037874 10336283.690309  4363370.978199  5115467.421334
  4338797.38985   5498009.649074  9326328.704528  5923465.518251
  5385066.084834 14681075.823981  4441411.462692 11608469.414189
  7295246.219064  5288191.255319 15034797.190652  6133897.404852
  9708663.109911  9578684.446446] [7509575.112959 8386197.194586 7367321.971846 7727207.847114
 7623229.149513 7517592.475681 7019180.20164  7358919.300135
 7393052.276302 5978457.233783 7310575.608059 6225324.694246
 7969485.988439 7560300.299773 6738744.249503 6715652.800253
 5627857.889076 7079889.64902  7047651.073445 7730080.274383
 7228717.815621 5329702.986404 7888247.088016 6852067.970702
 7791471.965063 5309667.242332 6184532.145898 6761796.988949
 7131290.449175 7438118.577761]


# Plotting 

In [438]:
def plot_map():
    p = figure(x_range=(3e6, 16e6), y_range=(3000000, 10000000),
               x_axis_type="mercator", y_axis_type="mercator")

    p.add_tile(CARTODBPOSITRON)

    source = ColumnDataSource(data=dict(longitude=x, latitude=y, names=df_city.Город))

    p.circle(x='longitude', y='latitude', source=source)

    labels = LabelSet(x='longitude', y='latitude', text='names', level='glyph',
                  x_offset=5, y_offset=5, source=source, render_mode='canvas')

    p.add_layout(labels)

    output_notebook()

    show(p)

In [436]:
def plot_map_with_path(df):
    
    project_projection = pyproj.Proj("+init=EPSG:4326")  # wgs84
    google_projection = pyproj.Proj("+init=EPSG:3857")  # default google projection

    longitude =  df.Долгота.values
    latitude = df.Широта.values

    x, y = pyproj.transform(project_projection, google_projection, longitude, latitude)

    p = figure(x_range=(3e6, 16e6), y_range=(3000000, 10000000),
               x_axis_type="mercator", y_axis_type="mercator")

    p.add_tile(CARTODBPOSITRON)

    source = ColumnDataSource(data=dict(longitude=x, latitude=y, names=df.Город))

    p.circle(x='longitude', y='latitude', source=source)

    glyph = Line(x="longitude", y="latitude", line_color="#f46d43", line_width=4, line_alpha=0.8)
    p.add_glyph(source, glyph)
    
    labels = LabelSet(x='longitude', y='latitude', text='names', level='glyph',
                  x_offset=5, y_offset=5, source=source, render_mode='canvas')

    p.add_layout(labels)

    output_notebook()

    show(p)

# SA

## Distance

In [137]:
def distance(a, b):
    R = 6373.0
    
    lon1 = radians(a.Долгота)
    lon2 = radians(b.Долгота)
    lat1 = radians(a.Широта)
    lat2 = radians(b.Широта)
    
    
    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    return R * c

In [140]:
def path_distance(x):
    fin_dist = 0
    prev_elemet = x.iloc[-1]
    for el in x.itertuples():
        fin_dist += distance(prev_elemet, el)
        prev_elemet = el
    return fin_dist

## Algorithm 

In [162]:
def p(x0,T):
    return np.exp(-path_distance(x0)/T)

In [482]:
def gen_x(df):
    x_temp = df.copy()
    
    a = rand.randint(df_city.shape[0])
    b = rand.randint(df_city.shape[0])
    
    temp = x_temp.iloc[a].copy()

    x_temp.iloc[a] = x_temp.iloc[b]
    x_temp.iloc[b] = temp
    
    return x_temp

In [483]:
def a_ratio(x0,x1, T):
    return np.exp((-path_distance(x1)/T) - (-path_distance(x0)/T))

In [488]:
x0 = df_city.copy()
plot_map_with_path(x0)
annealing_rate = 0.99
rand = np.random
print(path_distance(x0))


# for i in range(0, 60):
T = 25000
rand.seed(24)
while T>=1:
    new_x = gen_x(x0)
    a = a_ratio(x0,new_x, T)

    u = rand.sample()
    if u < a:
        x0 = new_x

    T *= annealing_rate
print(path_distance(new_x))

plot_map_with_path(new_x)

Loading BokehJS ...

65189.55479956974
20356.210832827954


Loading BokehJS ...

T = 25000
ann_rate = 0.99
seed = 21, 24

T = 40000
ann_rate = 0.99
seed = 19, 24, 45

T = 65000
ann_rate = 0.99
seed = 4, 19

# Results 

In [491]:
print('x0 distance: %lf'%(path_distance(df_city)))
print('distance after SA: %lf'%(path_distance(new_x)))

x0 distance: 65189.554800
distance after SA: 20356.210833


In [490]:
plot_map_with_path(df_city)
plot_map_with_path(new_x)

Loading BokehJS ...

Loading BokehJS ...